In [7]:
from huggingface_hub import hf_hub_url, cached_download
import joblib
import pandas as pd

REPO_ID = "julien-c/wine-quality"
FILENAME = "sklearn_model.joblib"


model = joblib.load(cached_download(
    hf_hub_url(REPO_ID, FILENAME)
))


/home/dorian/.pyenv/versions/3.10.4/lib/python3.10/site-packages/huggingface_hub/file_download.py:672: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(
/home/dorian/.pyenv/versions/3.10.4/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/dorian/.pyenv/versions/3.10.4/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator SVC from version 0.24.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#secu

In [29]:
import mlflow.pyfunc
from zipfile import ZipFile
import os
import shutil
import datetime



group = "io.models"
name = "wine"
version = "1.0"

PUNCH_MODEL_CREATION_DIR = f'/tmp/punch_artifact_env/python/{group.replace(".", "/")}/{name}/{version}/'

shutil.rmtree(PUNCH_MODEL_CREATION_DIR, ignore_errors=True)
os.makedirs(PUNCH_MODEL_CREATION_DIR, exist_ok=True)

class ModelWrapper(mlflow.pyfunc.PythonModel):

    def __init__(self, model):
        self.model = model

    def predict(self, context, model_input):
        return self.model.predict(model_input)

# Supposez que `model` est votre modèle scikit-learn
model_wrapper = ModelWrapper(model)

# Maintenant, sauvegardez le wrapper
mlflow.pyfunc.save_model(python_model=model_wrapper, path=PUNCH_MODEL_CREATION_DIR)

with ZipFile(f'{PUNCH_MODEL_CREATION_DIR}/{name}_{version}.zip', 'w') as zipObj:
    for folderName, subfolders, filenames in os.walk(PUNCH_MODEL_CREATION_DIR):
        for filename in filenames:
            if filename != f'{name}_{version}.zip':
                filePath = os.path.join(folderName, filename)
                zipObj.write(filePath, filename)

# create metadata file
metadata = (f'type: model\n'
            f'group: {group}\n'
            f'logo: https://punchplatform.com/wp-content/uploads/2020/01/logos_png_logo_punch_color2019_marge.png\n'
            f'version: {version}\n'
            f'artifact: {name}\n'
            f'displayName: {name}\n'
            f'createdAt: {int(datetime.datetime.timestamp(datetime.datetime.now()) * 1000)}\n'
            f'description: model')

with open(f'{PUNCH_MODEL_CREATION_DIR}/metadata.yml', 'w') as f:
    f.write(metadata)
    
# make artifact zip
with ZipFile(f'{PUNCH_MODEL_CREATION_DIR}/artifact_{name}_{version}.zip', 'w') as zipObj:
    zipObj.write(f'{PUNCH_MODEL_CREATION_DIR}/{name}_{version}.zip', f'{name}_{version}.zip')
    zipObj.write(f'{PUNCH_MODEL_CREATION_DIR}/metadata.yml', 'metadata.yml')


In [31]:
ppf_get_model = "io.models:wine:1.0"
group, artifact, version = ppf_get_model.split(":")
model_path = f'tmp/punch_artifact_env/python/{group.replace(".", "/")}/{artifact}/{version}/'
print(model_path)
output = model_path

tmp/punch_artifact_env/python/io/models/wine/1.0/
